# Data Collection
This notebook is dedicated to the collection and insertion of data into a MySQL database hosted on Google Cloud. The collection code will ultimately be transferred to a python module file that will be saved in the same subfolder as this document.

In [1]:
import config
import mysql.connector 
import googlemaps
from datetime import datetime
from mysql.connector import errorcode

%load_ext autoreload
%autoreload 2



## Google Maps Directions API

API documentation available [here](https://developers.google.com/maps/documentation/directions/start?hl=en_US).

Python client for maps API available [here](https://github.com/googlemaps/google-maps-services-python)

In [22]:
gmaps = googlemaps.Client(key=config.api_key)

# Geocoding my address
geocode_my_result = gmaps.geocode(config.my_address)

# Store just latitude and longitude
my_geometry = geocode_my_result[0]['geometry']['location']

# Geocoding my girlfriend's address
geocode_gf_result = gmaps.geocode(config.gf_address)

# Store just latitude and longitude
gf_geometry = geocode_gf_result[0]['geometry']['location']

In [66]:
class GDirections:
    """
    A class to keep track of and display to the user only necessary parts of the Google maps searches.
    """

    
    def __init__(self, start_geometry, end_geometry, mode):
        
        """
        Parameters
        -----------
    
        start_geometry: latitude and longitude of start location
        end_geometry: latitude and longitude of end location
        mode: One of "driving", "walking", "bicycling" or "transit"
        """
        
        self._trip_start = datetime.now()
        self._directions_json = self._retrieve_google_directions_json(start_geometry, end_geometry, mode)
        self._trip_duration = self._calculate_trip_duration()
        
        
    def _retrieve_google_directions_json(self,start_geometry, end_geometry, mode):
        
        """
        Call Google Maps API to retrieve a json with directions information
        """
        
        # Call API
        directions = gmaps.directions(
            origin = start_geometry,
            destination = end_geometry,
            mode = mode,
            departure_time = self._trip_start
        )
        return directions
    
    def _calculate_trip_duration(self):
        
        """
        Calculate and return trip duration in minutes.
        
        Returns
        -------
        
        Trip duration as a string in minutes.
        
        e.g. '50 minutes'
        """
        
        # Store arrival time and convert to datetime
        arrival_time = self._directions_json[0]['legs'][0]['arrival_time']['text']
        arrival_datetime = datetime.strptime(arrival_time, '%H:%M%p')
    

        # Store arrival time and convert to datetime
        departure_time = self._directions_json[0]['legs'][0]['departure_time']['text']
        departure_datetime = datetime.strptime(departure_time, '%H:%M%p')
        print()
        # Calculate difference in seconds, convert to minutes
        trip_duration = int( (arrival_datetime - departure_datetime).seconds/60 )
        
        return trip_duration
    
    def get_trip_duration(self):
        """
        Getter method for trip_duration in minutes as int
        """
        
        return self._trip_duration
    
    def get_trip_instructions(self):
        """
        RETURNS
        -------
        A dictionary with which trains/buses to take in this trip and how long each one will take.

        PARAMETERS
        ----------

        directions: [JSON] raw directions contained within self._directions_json
        """
        # Empty container for results
        directions_list = []
    
        # Isolate just the specific trip directions
        directions = self._directions_json[0]['legs'][0]['steps']

        for direction in directions:
            
            # Ignore any walking instructions
            if direction.get('transit_details'):
                
                # Store name of bus or train line
                travel_mode = direction['transit_details']['line']['short_name']
                
                # Store time on that bus or train line in minutes as int
                direction_len = int(direction['duration']['text'].split()[0])
                
                # Append dict to the results container
                directions_list.append({
                    'travel_mode' : travel_mode,
                    'length' : direction_len
                })

        return directions_list
    
    def get_trip_start(self):
        return self._trip_start.strftime("%Y/%m/%d %H:%M")

In [67]:
# Trips to gf's home
transit_to_gf = GDirections(my_geometry, gf_geometry, 'transit')

# Trips to my home
transit_to_me = GDirections(gf_geometry, my_geometry, 'transit')

# Store these together in a list
trip_objects = [transit_to_gf, transit_to_me]

In [52]:
trip_objects[0].get_trip_start()

'2020/02/15 22:38'

In [74]:
arrival_time = trip_objects[0]._directions_json[0]['legs'][0]['arrival_time']['text']
arrival_datetime = datetime.strptime(arrival_time, '%H:%M%p')

In [72]:
departure_time = trip_objects[0]._directions_json[0]['legs'][0]['departure_time']['text']
departure_datetime = datetime.strptime(departure_time, '%H:%M%p')

# The problem is that there is the case of overnight or at midday. Note that even though we're going from 12 - 1 am, the day is shown as 01-01 either way. Need to keep track of the current day and use that in the delta calculation. Then will need to figure out automation once that's done.

In [83]:
print(arrival_time, arrival_datetime)
print(departure_time, departure_datetime)

1:00am 1900-01-01 01:00:00
12:18am 1900-01-01 12:18:00


In [75]:
arrival_datetime

datetime.datetime(1900, 1, 1, 1, 0)

In [76]:
departure_datetime

datetime.datetime(1900, 1, 1, 12, 18)

In [79]:
delta = departure_datetime - arrival_datetime #- departure_datetime

In [82]:
delta.seconds/60

678.0

In [68]:
trip_objects[0].get_trip_duration()

762

In [65]:
trip_objects[0].get_trip_instructions()

[{'travel_mode': 'Q59', 'length': 21}, {'travel_mode': 'Q60', 'length': 11}]

In [27]:
# #Convert date string back to datetime
# #This will be used to convert the string back to datetime object after we pull back down from db

# datetime.strptime(db_time, "%Y/%m/%d %H:%M")

# Set up MySQL database
Automate data collection to add trip information every 5 mins or so.

In [12]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)
cursor = cnx.cursor()

cursor.close()
cnx.close()

In [18]:
## Create new DB 
db_name = 'google_maps'

def create_database(cursor, db_name):
    """
    Creates a MySQL database.
    
    PARAMETERS
    ----------
    
    cursor: cursor for a given mysql connection object
    db_name: [str] name of the new database
    """
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(db_name))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))


try:
    # Check to see if db exists
    cursor.execute("USE {}".format(db_name))
    
except mysql.connector.Error as err:
    
    print("Database {} does not exists.".format(db_name))
    
    # If db doesn't exist, create it
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        
        create_database(cursor, db_name)
        
        print("Database {} created successfully.".format(db_name))
        
        cnx.database = db_name
    
    # If another error, print it
    else:
        print(err)

Database google_maps does not exists.
Database google_maps created successfully.


In [61]:
# Create a table for the data

create_query = """
CREATE TABLE trips (
    departure_time varchar(20) NOT NULL,
    trip_direction int(5) NOT NULL,
    trip_duration int(10) NOT NULL,
    trip_instructions varchar(150) NOT NULL,
    PRIMARY KEY (departure_time, trip_direction)
    );

"""
cursor.execute(create_query)

In [60]:
# write a function to insert data into the reviews table
def insert_data(trip_objects):
    """
    Inserts data into google_maps MySQL db.
    
    PARAMETERS
    ----------
    
    trip_object: List of 2 GDirections trip objects containing information 
                 to be inserted into the MySQL table. First object contains
                 info from my house to gf and second object is the other trip.
    """
    # Create list of tuples for the objects in trip_objects
    trip_tuples = [
        (
        trip_object.get_trip_start(),         # departure_time
        i,                                    # trip_direction (either 0 or 1)
        trip_object.get_trip_duration(),      # trip_duration
        str(trip_object.get_trip_instructions())   # trip_instructinos
        )
        for i, trip_object in enumerate(trip_objects)
    ]
       
    insert_statement = """
                       INSERT INTO 
                         trips (departure_time, trip_direction, trip_duration, trip_instructions) 
                       VALUES 
                         (%s, %s, %s, %s)
                       ;"""
    cursor.executemany(insert_statement,trip_tuples)
    cnx.commit()

In [59]:
insert_data(trip_objects)

[('2020/02/15 22:38', 0, 49, "[{'travel_mode': 'Q59', 'length': 23}, {'travel_mode': 'R', 'length': 5}]"), ('2020/02/15 22:38', 1, 62, "[{'travel_mode': 'Q23', 'length': 10}, {'travel_mode': 'Q54', 'length': 21}]")]
